In [ ]:
import csv
!pip install wandb
import torch
import heapq
import torch.nn as nn
import random
import numpy as np
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt55
import matplotlib.pyplot as plt
import torch.nn.functional as F
import matplotlib.ticker as ticker
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=1a8b5cfd03608310c51cd7204255564c3647b104a0dbb2e3d33ecd25f57f8522
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb
wandb.login(key='b032cc059132c9aac4f1b317f6f9ad007ef9e4d4')
wandb.init(project="Assignment3_final",name="Question_5")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m083. Use `wandb login --relogin` to force relogin


In [ ]:
# to mount to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/aksharantar_dataset/tel/'

/content/drive/MyDrive/aksharantar_dataset/tel


In [ ]:
val_tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_valid.csv")
tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_train.csv")
test_tsv_file = open("/content/drive/MyDrive/aksharantar_dataset/tel/tel_test.csv")
val_read_tsv = csv.reader(val_tsv_file)
read_tsv = csv.reader(tsv_file)
test_read_tsv = csv.reader(test_tsv_file)

In [ ]:
train_Y, train_X = [], []

for i in read_tsv:   
    train_Y.append(i[1])
    train_X.append(i[0])

test_Y, test_X = [], []
for i in test_read_tsv:
    test_Y.append(i[1])
    test_X.append(i[0])

val_Y, val_X = [], []
for i in val_read_tsv:
    val_Y.append(i[1])
    val_X.append(i[0])

train_Y, train_X = np.array(train_Y), np.array(train_X)
for i in range(train_Y.shape[0]):
    train_Y[i] = "\t" + train_Y[i] + "\n"

test_Y, test_X = np.array(test_Y), np.array(test_X)
for i in range(test_Y.shape[0]):
  test_Y[i] = "\t" + test_Y[i] + "\n"

val_Y, val_X = np.array(val_Y), np.array(val_X)
for i in range(val_Y.shape[0]):
   val_Y[i] = "\t" + val_Y[i] + "\n"
ch_end, c_end =  " ", " "

In [ ]:
val_output_corpus, output_corpus, input_corpus, val_input_corpus = set(), set(), set(), set()
d_type, td_type = "int64", torch.int64

for w in val_Y:
    for char in w:
        if char not in val_output_corpus:
            val_output_corpus.add(char)

for w in train_Y:
    for char in w:
        if char not in output_corpus:
            output_corpus.add(char)

for w in val_X:
    for char in w:
        if char not in val_input_corpus:
            val_input_corpus.add(char)

output_corpus.add(ch_end)

for w in train_X:
    for char in w:
        if char not in input_corpus:
            input_corpus.add(char)


In [ ]:
input_corpus.add(ch_end)
t_len = len(train_X)
output_corpus, input_corpus = sorted(list(output_corpus)), sorted(list(input_corpus))
num_decoder_tokens, num_encoder_tokens = len(output_corpus), len(input_corpus)
max_decoder_seq_length, max_encoder_seq_length = max([len(txt) for txt in train_Y]),  max([len(txt) for txt in train_X]) + 2
v_len = len(val_X)

In [ ]:
print("Max sequence length for outputs:", max_decoder_seq_length)
print("Number of unique input tokens:", num_encoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Number of unique output tokens:", num_decoder_tokens)
print("Number of samples:", len(train_X))

Max sequence length for outputs: 21
Number of unique input tokens: 27
Max sequence length for inputs: 30
Number of unique output tokens: 65
Number of samples: 51200


In [ ]:
output_char_index, input_char_index = dict([(char, i) for i, char in enumerate(output_corpus)]), dict([(char, i) for i, char in enumerate(input_corpus)])


target_data, input_data = np.zeros((max_decoder_seq_length,t_len), dtype = d_type), np.zeros((max_encoder_seq_length,t_len), dtype = d_type)
t_z = 0 
for i, (x, y) in enumerate(zip(train_X, train_Y)):
    for t, char in enumerate(x):
        input_data[t, i] = input_char_index[char]
    t_z = t+1    
    input_data[t_z :,i] = input_char_index[ch_end]
    t =  0
    for t, char in enumerate(y):
        target_data[t, i] = output_char_index[char]
    t_z = t+1       
    target_data[t_z :,i] = output_char_index[ch_end]
    
target_data_val, input_data_val = np.zeros((max_decoder_seq_length,v_len), dtype= d_type), np.zeros((max_encoder_seq_length,v_len), dtype= d_type)
t_z = 0
for i, (x, y) in enumerate(zip(val_X, val_Y)):
    for t, char in enumerate(x):
        t_z = 0
        input_data_val[t, i] = input_char_index[char]
    t_z = t+1   
    input_data_val[t_z :,i] = input_char_index[ch_end]
    t_z = 0
    for t, char in enumerate(y):
        target_data_val[t, i] = output_char_index[char]
    t_z = t+1       
    target_data_val[t_z :,i] = output_char_index[ch_end]

In [ ]:
# convertin numpy arrays to tensors
target_data, input_data =  torch.tensor(target_data,dtype = td_type),  torch.tensor(input_data,dtype = td_type)
target_data_val, input_data_val = torch.tensor(target_data_val,dtype = td_type), torch.tensor(input_data_val,dtype = td_type)

**Adding an attention network to my base sequence-to-sequence model**

In [ ]:
output_size, input_size_decoder, input_size_encoder = num_decoder_tokens, num_decoder_tokens, num_encoder_tokens
dec_dropout, enc_dropout = 0.1, 0.1
num_epochs, batch_size, learning_rate= 10,32,0.001
load_model, training = False, False
num_dec_layers, num_enc_layers = 1, 1
decoder_embedding_size, encoder_embedding_size, hidden_size = 256, 256, 256 # Needs to be the same for both RNN's

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_target_char_index, reverse_input_char_index = dict((i, char) for char, i in output_char_index.items()), dict((i, char) for char, i in input_char_index.items())

**LSTM**

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        drop_par, emb_size = dropout, embedding_size
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, inp, self.hidden_size = num_layers, input_size, hidden_size
        lstm_par = hidden_size * 2 + emb_size
        self.embedding = nn.Embedding(inp, emb_size)
        lin_par = hidden_size * 3
        self.rnn = nn.LSTM(lstm_par, hidden_size, num_layers)
        
        self.energy = nn.Linear(lin_par, 1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax( dim = 0)
        self.relu = nn.Hardshrink()

    def forward(self, x, encoder_states, hidden, cell):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x, _dim  = x.unsqueeze(0), 2
        drop_par, sequence_length = self.embedding(x), encoder_states.shape[0]
        embedding = self.dropout(drop_par)
        # embedding shape: (1, N, embedding_size)
        
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)
        relu_par = torch.cat((h_reshaped, encoder_states), dim = _dim)
        energy = self.relu(self.energy(relu_par))
        # energy: (seq_length, N, 1)
        
        attention = self.softmax(energy)
        # attention: (seq_length, N, 1)
        
        encoder_states = encoder_states.permute(1,0,2)

        attention = attention.permute(1,2,0)
        # attention: (N, seq_length, 1)

        encoder_states = self.relu(encoder_states)
        # encoder_states: (N, seq_length, hidden_size*2)
        context_init = torch.bmm(attention, encoder_states)
        context_vector = context_init.permute(1,0,2)
        
        rnn_input = torch.cat((context_vector,embedding),dim = _dim)
       # rnn_input = self.relu(rnn_input)

        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # outputs shape: (1, N, hidden_size)
        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = self.fc(outputs).squeeze(0) #predictions = self.fc(outputs)

        return predictions, hidden, cell, attention

In [ ]:

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.decoder, self.encoder = decoder, encoder
  
    def forward(self, source, target, teacher_force_ratio = 0.5 ): #teacher_force_ratio hyper_parameter

        target_len, target_vocab_size, batch_size = target.shape[0], num_decoder_tokens, source.shape[1]
        encoder_states, hidden, cell = self.encoder(source)
        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell, _ = self.decoder(x, encoder_states, hidden, cell)

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess, ind = output.argmax(1), t

            # Store next output prediction
            outputs[ind] = output

            x = best_guess if random.random() >= teacher_force_ratio else  target[t]

        return outputs

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        drop_par, emb_size, inp, fc_par = dropout, embedding_size, input_size, (hidden_size * 2)
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, self.hidden_size, bidir = num_layers, hidden_size, True

        self.embedding = nn.Embedding(inp, emb_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional = bidir)
        
        self.fc_hidden = nn.Linear(fc_par, hidden_size)
        self.fc_cell = nn.Linear(fc_par, hidden_size)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        drop_par, fc_dim = self.embedding(x), 2
        embedding = self.dropout(drop_par)
        # embedding shape: (seq_length, N, embedding_size)
        encoder_states, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        hidden_par = torch.cat((hidden[0:1], hidden[1:2]), dim = fc_dim)
        cell_par = torch.cat((cell[0:1], cell[1:2]), dim = fc_dim)
        hidden = self.fc_hidden(hidden_par)
        cell = self.fc_cell(cell_par)

        return encoder_states, hidden, cell

**RNN**

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderRNN, self).__init__()
        drop_par, emb_size = dropout, embedding_size
        self.dropout = nn.Dropout(dropout)
        self.num_layers, self.hidden_size, inp = num_layers, hidden_size, input_size
        rnn_par =  hidden_size * 2 + emb_size
        self.embedding = nn.Embedding(inp, emb_size)
        rnn_lin_par, soft_dim = hidden_size * 3, 0
        self.rnn = nn.RNN(rnn_par, hidden_size, num_layers)
        self.energy = nn.Linear(rnn_lin_par, 1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim = soft_dim)
        self.relu = nn.Hardshrink()

    def forward(self, x, encoder_states, hidden):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x, relu_dim = x.unsqueeze(0), 2
        sequence_length, drop_par = encoder_states.shape[0], self.embedding(x)
        embedding = self.dropout(drop_par)
        # embedding shape: (1, N, embedding_size)
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)
        relu_par = torch.cat((h_reshaped, encoder_states), dim = relu_dim)
        energy = self.relu(self.energy(relu_par))
        # energy: (seq_length, N, 1)
        
        attention = self.softmax(energy)
        # attention: (seq_length, N, 1)
        encoder_states = encoder_states.permute(1,0,2)

        attention = attention.permute(1,2,0)
        # attention: (N, seq_length, 1)

        encoder_states = self.relu(encoder_states)
        # encoder_states: (N, seq_length, hidden_size*2)
        context_init = torch.bmm(attention, encoder_states)
        context_vector = context_init.permute(1,0,2)
        
        rnn_input = torch.cat((context_vector, self.dropout(drop_par)),dim = relu_dim)
       # rnn_input = self.relu(rnn_input)
        outputs, (hidden) = self.rnn(rnn_input, (hidden))
        # outputs shape: (1, N, hidden_size)

        #predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = self.fc(outputs).squeeze(0)

        return predictions, hidden, attention
class Seq2SeqGR(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqGR, self).__init__()
        self.decoder, self.encoder = decoder, encoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        target_len, target_vocab_size, batch_size = target.shape[0], num_decoder_tokens, source.shape[1]

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        
        # Grab the first input to the Decoder which will be <SOS> token
        x , n_len = target[0], target_len
        encoder_states, hidden = self.encoder(source)

        for t in range(1, n_len):
            # Use previous hidden, cell as context from encoder at start
            
            output, hidden, _ = self.decoder(x, encoder_states, hidden)

            # Get the best word the Decoder predicted (index in the vocabulary)
            ind, best_guess = t, output.argmax(1)

            # Store next output prediction
            outputs[ind] = output

            x = best_guess if random.random() >= teacher_force_ratio else target[t]

        return outputs

class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(EncoderRNN, self).__init__()
        drop_par, emb_size = dropout, embedding_size
        self.dropout = nn.Dropout(dropout)
        self.num_layers, self.hidden_size, inp, bid_dec = num_layers, hidden_size, input_size, True
        self.embedding = nn.Embedding(input_size, emb_size)
        fc_lin_par = hidden_size * 2
        self.rnn = nn.RNN(emb_size, hidden_size, num_layers, bidirectional = bid_dec)
        self.fc_hidden = nn.Linear(fc_lin_par, hidden_size)
        self.fc_cell = nn.Linear(fc_lin_par, hidden_size)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        drop_par = self.embedding(x)
        embedding = self.dropout(drop_par)
        # embedding shape: (seq_length, N, embedding_size)

        encoder_states, hidden = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        fc_hidden_par, fc_dim = (hidden[0:1], hidden[1:2]), 2
        hidden = self.fc_hidden(torch.cat(fc_hidden_par, dim = fc_dim))
        #cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))

        return encoder_states, hidden

**GRU**

In [ ]:
class DecoderGRU(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderGRU, self).__init__()
        drop_par, emb_size = dropout, embedding_size
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, self.hidden_size, inp = num_layers, hidden_size, input_size
        gru_par = hidden_size * 2 + emb_size
        self.embedding = nn.Embedding(inp, emb_size)
        self.rnn = nn.GRU(gru_par, hidden_size, num_layers)
        lin_par, s_dim = hidden_size * 3, 0
        self.energy = nn.Linear(lin_par, 1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim = s_dim)
        self.relu = nn.Hardshrink()

    def forward(self, x, encoder_states, hidden):
        # x shape: (N) where N is for batch size, we want it to be (1, N), seq_length
        # is 1 here because we are sending in a single word and not a sentence
        x = x.unsqueeze(0)
        drop_par, sequence_length = self.embedding(x), encoder_states.shape[0]
        r_dim = 2
        embedding = self.dropout(drop_par)
        # embedding shape: (1, N, embedding_size)
        
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)
        relu_par = torch.cat((h_reshaped, encoder_states), dim = r_dim)
        energy = self.relu(self.energy(relu_par))
        # energy: (seq_length, N, 1)
        
        attention = self.softmax(energy)
        # attention: (seq_length, N, 1)
        encoder_states = encoder_states.permute(1,0,2)

        attention = attention.permute(1,2,0)
        # attention: (N, seq_length, 1)
        encoder_states = self.relu(encoder_states)
        # encoder_states: (N, seq_length, hidden_size*2)
        context_init = torch.bmm(attention, encoder_states)
        context_vector = context_init.permute(1,0,2)
        
        rnn_input = torch.cat((context_vector, self.dropout(drop_par)),dim = r_dim)
       # rnn_input = self.relu(rnn_input)
        outputs, (hidden) = self.rnn(rnn_input, (hidden))
        # outputs shape: (1, N, hidden_size)

        #predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = self.fc(outputs).squeeze(0)

        return predictions, hidden, attention
class Seq2SeqGR(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqGR, self).__init__()
        self.decoder, self.encoder = decoder, encoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        target_len, target_vocab_size, batch_size = target.shape[0], num_decoder_tokens, source.shape[1]
        
        encoder_states, hidden = self.encoder(source)

         # Grab the first input to the Decoder which will be <SOS> token
        n_len, x = target.shape[0], target[0]
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        for t in range(1, n_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, _ = self.decoder(x, encoder_states, hidden)

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess, ind = output.argmax(1), t

            # Store next output prediction
            outputs[ind] = output

            x = best_guess if random.random() >= teacher_force_ratio else target[t]

        return outputs

class EncoderGRU(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(EncoderGRU, self).__init__()
        drop_par, emb_size, bidir = dropout, embedding_size, True
        self.dropout = nn.Dropout(drop_par)
        self.num_layers, self.hidden_size, inp = num_layers, hidden_size, input_size
        self.embedding = nn.Embedding(inp, emb_size)
        fc_lin_par = hidden_size * 2
        self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, bidirectional = bidir)
        
        self.fc_hidden = nn.Linear(fc_lin_par, hidden_size)
        self.fc_cell = nn.Linear(fc_lin_par, hidden_size)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size
        drop_par, fc_dim  = self.embedding(x), 2
        embedding = self.dropout(drop_par)
        # embedding shape: (seq_length, N, embedding_size)
        
        encoder_states, hidden = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)
        fc_hidden_par = (hidden[0:1], hidden[1:2])
        fc_par = torch.cat(fc_hidden_par, dim = fc_dim)
        hidden = self.fc_hidden(fc_par)
        #cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))

        return encoder_states, hidden

In [ ]:
hidden_size = 128  # Needs to be the same for both RNN's
dec_dropout, enc_dropout = 0.8, 0.8
decoder_embedding_size, encoder_embedding_size = 128, 128
num_layers, output_size = 1, num_decoder_tokens
input_size_decoder, input_size_encoder = num_decoder_tokens, num_encoder_tokens

In [ ]:
def beam_searchGR(model, word, input_char_index, output_char_index, reverse_input_char_index,
                reverse_target_char_index, max_encoder_seq_length, max_decoder_seq_length,
                num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):

    # Encode the input word
    data, word_t = np.zeros((max_encoder_seq_length, 1), dtype= d_type), ''
    t_z, tab_pad, nl_pad,prob_dim, bw = 0, '\t', '\n', 1, beam_width
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1
    data[t_z :, 0] = input_char_index[c_end]

    data = torch.tensor(data, dtype= td_type).to(device)

    with torch.no_grad():
        encoder_states, hidden = model.encoder(data)

    # Initialize beam
    out_t = output_char_index[tab_pad]
    out_reshape = np.array(out_t).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]  # [(score, sequence, hidden)]

    for _ in range(max_decoder_seq_length):
      score_cal , candidates = 0, []
      for score, seq, hidden in beam:
          score_cal = score_cal + score
          last_token = seq[-1].item()
          if last_token == output_char_index[nl_pad]:
              # If the sequence ends with the end token, add it to the candidates
              candidates.append((score, seq, hidden))
              
              continue
          
          last_t_reshape = np.array(last_token).reshape(1,)
          hidden_par = hidden.squeeze(0)
          x = torch.tensor(last_t_reshape).to(device)
          output, hidden,at = model.decoder(x,encoder_states, hidden_par)
          probabilities = F.softmax(output, dim = prob_dim)

          # Get the top-k probabilities and tokens
          topk_probs, topk_tokens = torch.topk(probabilities, k= bw)
          cal_score = score
          for prob, token in zip(topk_probs[0], topk_tokens[0]):
              cal_score = cal_score + prob
              new_seq = torch.cat((seq, token.unsqueeze(0)), dim=0)
              n_hidden = hidden.clone()
              new_seq_ln  = len(new_seq)
              pen_fac = (new_seq_ln - 1) / 5
              new_hidden = n_hidden.unsqueeze(0)
              #length_penalty_factor = ((pen_fac) ** length_penalty)  # Adjust penalty factor as needed
              
              candidates.append((score + torch.log(prob).item() / ((pen_fac) ** length_penalty), new_seq, new_hidden))
              cal_score = 0
      # Select top-k candidates based on the accumulated scores
      beam = heapq.nlargest(beam_width, candidates, key = lambda x: x[0])

    
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0])# Select the best sequence from the beam as the output
    
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[1:-1]])

    return word_t

In [ ]:

def beam_search(model, word, input_char_index, output_char_index, reverse_input_char_index,reverse_target_char_index, max_encoder_seq_length, 
                max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens, beam_width, device, length_penalty=0.6):


    # Encode the input word
    data, word_t = np.zeros((max_encoder_seq_length, 1), dtype= d_type), ''
    t_z , bw = 0, beam_width
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1
    data[t_z:, 0] = input_char_index[c_end]

    data = torch.tensor(data, dtype= td_type ).to(device)

    with torch.no_grad():
        encoder_states,hidden,cell = model.encoder(data)

    # Initialize beam
    out_t = output_char_index['\t']
    out_reshape = np.array(out_t).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]  # [(score, sequence, hidden)]

    for _ in range(max_decoder_seq_length):
      candidates = []
      for score, seq, hidden in beam:
          # last_token = seq[-1].item()
          if seq[-1].item() == output_char_index['\n']:
              # If the sequence ends with the end token, add it to the candidates
              candidates.append((score, seq, hidden))
              continue
          lt_reshape = np.array(seq[-1].item()).reshape(1,) # last_token = seq[-1].item()
          hidden_upar = hidden.squeeze(0)
          x = torch.tensor(lt_reshape).to(device)
          output, hidden,cell,at = model.decoder(x, encoder_states, hidden_upar,cell)
          probabilities = F.softmax(output, dim=1)
          cal_cand = 0
          # Get the top-k probabilities and tokens
          topk_probs, topk_tokens = torch.topk(probabilities, k=bw)

          for prob, token in zip(topk_probs[0], topk_tokens[0]):
              cal_cand = prob
              n_hidden = hidden.clone()
              new_seq = torch.cat((seq, token.unsqueeze(0)), dim=0)
              ln_ns = len(new_seq)
              ln_pf = ((ln_ns - 1) / 5)
              new_hidden = n_hidden.unsqueeze(0)
              #length_penalty_factor = ln_pf ** length_penalty  # Adjust penalty factor as needed
              candidate_cal = score + torch.log(prob).item() / (ln_pf ** length_penalty)
              candidates.append((candidate_cal, new_seq, new_hidden))

      beam = heapq.nlargest(bw, candidates, key=lambda x: x[0])# Select top-k candidates based on the accumulated scores

      
    best_score, best_sequence, _ = max(beam, key=lambda x: x[0]) # Select the best sequence from the beam as the output
    
    cal_score = best_score
    word_t = ''.join([reverse_target_char_index[token.item()] for token in best_sequence[1:-1]])
    cal_score = 0
    return word_t

In [ ]:
hidden_size = 256  # Needs to be the same for both RNN's
load_model, training = False, False
output_size = num_decoder_tokens
num_epochs, batch_size, learning_rate = 2, 32, 0.001
dec_dropout, enc_dropout = 0.1, 0.1
encoder_embedding_size, decoder_embedding_size = 256, 256
num_dec_layers,num_enc_layers = 1, 1
input_size_encoder, input_size_decoder = num_encoder_tokens, num_decoder_tokens

In [ ]:
def training(num_encoder_tokens,input_embedding_size, dp, cell_type, hidden_size, num_enc_layers, num_dec_layers,num_epochs,output_size,input_size_decoder,batch_size,beam_width):
    #Decoder Selection
    if(cell_type=="LSTM"):
        decoder_net = Decoder(input_size_decoder,input_embedding_size,hidden_size,output_size,num_dec_layers,dp).to(device)
    elif(cell_type=="GRU"):
        decoder_net = DecoderGRU(input_size_decoder,decoder_embedding_size,hidden_size,output_size,num_dec_layers,dec_dropout).to(device)
    else:
        decoder_net = DecoderRNN(input_size_decoder,decoder_embedding_size,hidden_size,output_size,num_dec_layers,dec_dropout).to(device)
    #Encoder Selection
    if(cell_type=="LSTM"):
        encoder_net = Encoder(input_size_encoder,input_embedding_size, hidden_size, num_enc_layers,dp).to(device)
    elif(cell_type=="GRU"):
        encoder_net = EncoderGRU(input_size_encoder, encoder_embedding_size, hidden_size, num_enc_layers, enc_dropout).to(device)
    else:
        encoder_net = EncoderRNN(input_size_encoder, encoder_embedding_size, hidden_size, num_enc_layers, enc_dropout).to(device)
    
    #Model Selection
    if(cell_type=="LSTM"):    
        model = Seq2Seq(encoder_net, decoder_net).to(device)
    else:
        model = Seq2SeqGR(encoder_net, decoder_net).to(device)
    
    split_dim, bs,  m_norm = 1, batch_size, 1
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    train_ds_y, train_ds_x = torch.split(target_data, bs, dim = split_dim), torch.split(input_data, bs, dim = split_dim)
    correct_prediction  = 0
    #print(train_ds_x)
    for epoch in range(num_epochs):
        print(f"[Epoch {epoch} / {num_epochs}]")
        model.eval()
        total_count = len(val_X)
        model.train()
        for i, (x,y) in enumerate(zip(train_ds_x,train_ds_y)):
            # Get input and targets and get to cuda
            target, inp_data = y.to(device), x.to(device)
            # Forward prop
            output = model(inp_data, target)
            target, output = target[1:].reshape(-1),  output[1:].reshape(-1, output.shape[2])
            optimizer.zero_grad()
            loss = criterion(output, target)
            loss.backward() # Back prop
            # Clip to avoid exploding gradient issues, makes sure grads are
            # within a healthy range
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = m_norm)
            # Gradient descent step
            optimizer.step()
        correct_pred, total_words, lstm_bw = 0, total_count, 1
        model.eval()
        for i in range(total_count):
            if(cell_type=="LSTM"):
                decoded_sentence = beam_search(model,val_X[i], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index, 
                                                     max_encoder_seq_length, max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens,lstm_bw,device)
            else:
                 decoded_sentence = beam_searchGR(model,val_X[i], input_char_index, output_char_index, reverse_input_char_index, reverse_target_char_index, 
                                                        max_encoder_seq_length, max_decoder_seq_length,num_encoder_tokens, num_decoder_tokens,beam_width,device)
                 
            if decoded_sentence == val_Y[i][1:-1]:
                correct_pred = correct_pred + 1
        test_accuracy = correct_pred / total_words
        print(correct_pred / total_words)  # test_accuracy = correct_pred / total_words
        wandb.log({'val_accuracy_with_attention' : test_accuracy*100})

In [ ]:
sweep_config2 = {
    'method': 'bayes',
    'parameters': {
                    'Number_Epochs_at' :{
                      'values':[10,20,30,40]
                      },
                    'cell_type_value_at':{
                      'values': ['LSTM','GRU','RNN']
                      },
                    'beam_width_at':{
                      'values':[1,2,3,4,5]
                      },
                   'hidden_size_EncDec_at': {
                      'values': [128, 256, 512]
                      },
                   'dropout_EncDec_at': {
                      'values': [0.1, 0.2, 0.3, 0.4]
                      },
                   'embedding_size_input_at': {
                      'values': [128, 256, 512]
                      },
                   'beam_width_at':{
                      'values':[1,2,3,4,5]
                      },
                   'batch_size_value_at': {
                      'values': [128,256,512]
                      }
                  },
     'metric': {'goal': 'maximize', 'name': 'val_accuracy_with_attention'}           
 }

In [ ]:
def train():
    var1 = wandb.init()
    var2 = var1.config
     # var2 is a variable that holds and saves hyperparameters and inputs
    wandb.run.name = 'Cell_Type:-' + var2.cell_type_value_at + ', Batch_Size:-' + str(var2.batch_size_value_at) + ', Epochs:-' + str(var2.Number_Epochs_at) + ', Dropout:-' + str(var2.dropout_EncDec_at) + ', Beam_Search:-' + str(var2.beam_width_at) +', Embedding_Size:-' + str(var2.embedding_size_input_at) + ', Hidden_Size:-' + str(var2.hidden_size_EncDec_at) 
    # training(input_size_encoder ,var2.embedding_size_input_at, var2.dropout_EncDec_at, 'LSTM', var2.hidden_size_EncDec_at, 1,  1,5,num_decoder_tokens,num_decoder_tokens,var2.batch_size_value_at,var2.beam_width_at)
    training(input_size_encoder ,var2.embedding_size_input_at, var2.dropout_EncDec_at, var2.cell_type_value_at, var2.hidden_size_EncDec_at, 1,  1,var2.Number_Epochs_at,num_decoder_tokens,num_decoder_tokens,var2.batch_size_value_at,var2.beam_width_at)

In [ ]:
sweep_id = wandb.sweep(sweep_config2, project="Assignment3_final")
wandb.agent(sweep_id, train, count=25)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 2h1c2fdv
Sweep URL: https://wandb.ai/cs22m083/Assignment3_Q5/sweeps/2h1c2fdv


wandb: Agent Starting Run: 7nfx1cig with config:
wandb: 	Number_Epochs_at: 30
wandb: 	batch_size_value_at: 256
wandb: 	beam_width_at: 4
wandb: 	cell_type_value_at: LSTM
wandb: 	dropout_EncDec_at: 0.2
wandb: 	embedding_size_input_at: 128
wandb: 	hidden_size_EncDec_at: 256


[Epoch 0 / 30]
0.01611328125
[Epoch 1 / 30]
0.128173828125
[Epoch 2 / 30]
0.241943359375
[Epoch 3 / 30]
0.307373046875
[Epoch 4 / 30]
0.35546875
[Epoch 5 / 30]
0.38720703125
[Epoch 6 / 30]
0.400390625
[Epoch 7 / 30]
0.435302734375
[Epoch 8 / 30]
0.44140625
[Epoch 9 / 30]
0.429931640625
[Epoch 10 / 30]
0.459716796875
[Epoch 11 / 30]
0.47509765625
[Epoch 12 / 30]
0.48193359375
[Epoch 13 / 30]
0.490478515625
[Epoch 14 / 30]
0.48095703125
[Epoch 15 / 30]
0.481201171875
[Epoch 16 / 30]
0.501220703125
[Epoch 17 / 30]
0.49365234375
[Epoch 18 / 30]
0.49951171875
[Epoch 19 / 30]
0.507080078125
[Epoch 20 / 30]
0.504150390625
[Epoch 21 / 30]
0.51708984375
[Epoch 22 / 30]
0.4990234375
[Epoch 23 / 30]
0.501220703125
[Epoch 24 / 30]
0.502197265625
[Epoch 25 / 30]
0.508544921875
[Epoch 26 / 30]
0.513671875
[Epoch 27 / 30]
0.515625
[Epoch 28 / 30]
0.515869140625
[Epoch 29 / 30]
0.515625


val_accuracy_with_attention,▁▃▄▅▆▆▆▇▇▇▇▇██▇▇██████████████
val_accuracy_with_attention,51.5625


wandb: Agent Starting Run: xq0v2dvq with config:
wandb: 	Number_Epochs_at: 10
wandb: 	batch_size_value_at: 128
wandb: 	beam_width_at: 2
wandb: 	cell_type_value_at: GRU
wandb: 	dropout_EncDec_at: 0.3
wandb: 	embedding_size_input_at: 128
wandb: 	hidden_size_EncDec_at: 256


[Epoch 0 / 10]
0.23828125
[Epoch 1 / 10]
0.35400390625
[Epoch 2 / 10]
0.40283203125
[Epoch 3 / 10]
0.427978515625
[Epoch 4 / 10]
0.451416015625
[Epoch 5 / 10]
0.4677734375
[Epoch 6 / 10]
